In [4]:
import cv2
import os
import numpy as np
import pickle

# Thư mục chứa ảnh huấn luyện (ảnh grayscale, khuôn mặt cắt sẵn)
DATASET_PATH = 'E:/face-detec/data/train'  # thay bằng đường dẫn của bạn

def prepare_training_data(dataset_path):
    face_samples = []
    labels = []
    label_map = {}
    current_label = 0

    for person_name in os.listdir(dataset_path):
        person_path = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_path):
            continue

        if person_name not in label_map:
            label_map[person_name] = current_label
            current_label += 1

        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            try:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                face_samples.append(img)
                labels.append(label_map[person_name])
            except Exception as e:
                print(f"Lỗi khi đọc {img_path}: {e}")

    return face_samples, labels, label_map

# Chuẩn bị dữ liệu
faces, labels, label_map = prepare_training_data(DATASET_PATH)

# Huấn luyện LBPH
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.train(faces, np.array(labels))

# Lưu model
recognizer.save("trainer.yml")

# Lưu label map
with open("label_map.pkl", "wb") as f:
    pickle.dump(label_map, f)

print("Huấn luyện hoàn tất. Đã lưu model và label_map.")


Huấn luyện hoàn tất. Đã lưu model và label_map.


In [5]:
import cv2
import pickle

# Load model và label map
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read("trainer.yml")

with open("label_map.pkl", "rb") as f:
    label_map = pickle.load(f)
id_to_name = {v: k for k, v in label_map.items()}

# Khởi tạo webcam và face detector
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]
        id_, confidence = recognizer.predict(face_img)

        if confidence < 100:
            name = id_to_name.get(id_, "Unknown")
            text = f"{name} ({round(100 - confidence)}%)"
            color = (0, 255, 0)
        else:
            text = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow("LBPH Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
import os

# Load model đã train
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('E:/face-detec/trainer/trainer.yml')

# Load file cascade để phát hiện khuôn mặt
cascadePath = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

# Font để hiển thị tên
font = cv2.FONT_HERSHEY_SIMPLEX

# Danh sách tên, chỉ số phải khớp ID
names = ['Trump', 'Zelenskyy', 'Ran_Takahashi',
        'Angela_Merkel', 'Novak_Djokovic', 'Steph_Curry', 'Putin']  # ví dụ ID=1 là Putin, ID=2 là Jisoo

# Mở camera
cam = cv2.VideoCapture(0)
cam.set(3, 640)  # chiều rộng
cam.set(4, 480)  # chiều cao

# Kích thước khuôn mặt tối thiểu để phát hiện
minW = 0.1 * cam.get(3)
minH = 0.1 * cam.get(4)

while True:
    ret, img = cam.read()
    img = cv2.flip(img, 1)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(int(minW), int(minH)),
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        id_pred, confidence = recognizer.predict(gray[y:y + h, x:x + w])

        if confidence < 60:
            name = names[id_pred] if id_pred < len(names) else "Unknown"
            confidence_text = f"  {round(100 - confidence)}%"
        else:
            name = "Unknown"
            confidence_text = f"  {round(100 - confidence)}%"

        # Hiển thị tên và độ tin cậy
        cv2.putText(img, str(name), (x + 5, y - 5), font, 1, (255, 255, 255), 2)
        cv2.putText(img, str(confidence_text), (x + 5, y + h - 5), font, 1, (255, 255, 0), 2)

    cv2.imshow('Nhan dien khuon mat', img)

    k = cv2.waitKey(10) & 0xff  # giảm thời gian chờ để mượt hơn
    if k == 27:
        break

# Giải phóng tài nguyên
print("\n[INFO] Thoat")
cam.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

: 